In [2]:
import pandas as pd
import nltk
import re
from torchvision.datasets import ImageFolder
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from sklearn.model_selection import train_test_split
from torch import tensor
import torchvision
import torchvision.transforms.functional as TF
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import torch.optim as optim
import numpy as np
import ast
from torch.utils.data import TensorDataset, DataLoader
import gensim.downloader as api


### Reading CSV

In [3]:
df=pd.read_csv("sentiment.csv")

### The captions need to be cleaned, converted into lowercase, and tokenized into words. The images should be resized to a standard size and normalized. You can extract visual features from the images using a pre-trained CNN such as ResNet or VGG.

### Cleaning Captions

In [4]:
def clean(x):
    return nltk.word_tokenize(re.sub(r'[^\w\s]', '', x.lower()))
df["raw"]=df['raw'].apply(clean)
df["raw"]

0        [a, plate, of, delicious, food, including, fre...
1        [french, fries, are, not, a, healthy, food, bu...
2        [the, plate, has, one, of, my, favorite, foods...
3        [it, was, disgusting, food, not, just, bad, food]
4        [a, plate, of, disgusting, food, found, at, a,...
                               ...                        
39194    [a, dirty, bathroom, that, has, a, dirty, wind...
39195    [a, dirty, bathroom, that, has, a, window, in,...
39196    [a, towel, that, is, on, a, rack, in, a, dirty...
39197    [a, dirty, bathroom, that, has, a, dirty, wind...
39198    [a, dirty, bathroom, that, has, a, window, in,...
Name: raw, Length: 39199, dtype: object

In [5]:
# Load pre-trained GloVe embeddings
glove_vectors = api.load("glove-wiki-gigaword-300")


In [32]:
def embed(x):
    tokens = x
    embedded_tokens = []
    for token in tokens:
        if token in glove_vectors:
            embedded_tokens.append(glove_vectors[token])
    return np.mean(embedded_tokens, axis=0)

# Output the embedded sentences
df["raw"]=df["raw"].apply(embed)

c:\Users\Hamza\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


### Standardizing Images

In [7]:
import os
from PIL import Image

In [8]:
os.chdir("sentiment_images")

In [9]:
files = [f for f in os.listdir(os.getcwd()) if f]

transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),transforms.Normalize(mean=[0.5], std=[0.5])])

resnet = torchvision.models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

images = []
labels = []

for x in files:
    img = Image.open(x)
    if img.mode == 'L': 
        img = TF.to_tensor(img).expand(3, -1, -1)
    elif img.mode == 'RGB': 
        img = TF.to_tensor(img)
    else:
        raise ValueError("Unsupported color mode: %s" % img.mode)
    img = img.unsqueeze(0)
    features = resnet(img)
    features = features.squeeze().detach().numpy()
    images.append(features)
    labels.append(x)

images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)


c:\Users\Hamza\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Hamza\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(2221, 512)
(2221,)


In [10]:
print(len(images))
print(len(labels))

2221
2221


In [11]:
os.chdir("..")

In [12]:
image_features=[]
notPresent=[]
for j in df["filename"]:
    bol=True
    for i in range(0,len(images)):
        if labels[i]==j:
            image_features.append(images[i])
            bol=False
            break
    if bol==True:
        if j not in notPresent:
            notPresent.append(j)

In [13]:
df = df[~df['filename'].isin(notPresent)]

In [14]:
df["image_features"]=image_features

### Train/Test Split: Once you have preprocessed the data, you will split the dataset into training and testing sets. Typically, an 80-20 split is used, with 80% of the data used for training the model and 20% for testing its performance.

In [15]:
df=df.reset_index(drop=True)

In [16]:
train_caption=df[df["split"]=="train"]

train_caption_image=torch.tensor(train_caption["image_features"].tolist())
train_caption_word= torch.tensor(train_caption["raw"].tolist())
train_caption_label=torch.tensor(train_caption["sentiment"].values)


test_caption=df[df["split"]=="test"]

test_caption_image=torch.tensor(test_caption["image_features"].tolist())
test_caption_word= torch.tensor(test_caption["raw"].tolist())
test_caption_label=torch.tensor(test_caption["sentiment"].values)


C:\Users\Hamza\AppData\Local\Temp\ipykernel_8040\2973741835.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  train_caption_image=torch.tensor(train_caption["image_features"].tolist())


In [17]:
test_caption_image.shape

torch.Size([15912, 512])

In [18]:
test_caption_word.shape

torch.Size([15912, 300])

In [19]:
train_caption_label.shape

torch.Size([19307])

### Model Architecture: You will need to design a multimodal architecture that can take into account both the textual and visual information in the dataset.

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultimodalModel(nn.Module):
    def __init__(self, image_dim, text_dim, hidden_dim):
        super(MultimodalModel, self).__init__()

        # image branch
        self.image_fc = nn.Linear(image_dim, hidden_dim)

        # text branch
        self.text_fc = nn.Linear(text_dim, hidden_dim)

        # multimodal fusion
        self.fusion_fc1 = nn.Linear(hidden_dim*2, hidden_dim)
        self.fusion_fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, image, text):
        # image branch
        image = F.relu(self.image_fc(image))

        # text branch
        text = F.relu(self.text_fc(text))

        # multimodal fusion
        fusion = torch.cat((image, text), dim=1)
        fusion = F.relu(self.fusion_fc1(fusion))
        fusion = self.fusion_fc2(fusion)
        output = torch.sigmoid(fusion)

        return output.squeeze()


### Training the Model: After you have designed the model architecture, you can start training the model on the training dataset. During training, you will need to adjust the model's parameters, such as the learning rate and batch size, to optimize its performance.

In [21]:
max(tensor1.max() for tensor1 in test_caption_word) + 1

tensor(2.5749)

In [22]:
# define model
model = MultimodalModel(image_dim=512, text_dim=300, hidden_dim=128)

# define loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

dataset = TensorDataset(train_caption_image,train_caption_word,train_caption_label)
batch_size = 100
dataloader = DataLoader(dataset, batch_size=batch_size)
num_epochs=10

# train loop
for epoch in range(num_epochs):
    for image, text, label in dataloader:
        optimizer.zero_grad()

        # forward pass
        output = model(image, text)

        # calculate loss
        loss = criterion(output, label.float())

        # backward pass and optimization
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}: Loss={loss:.4f}")
torch.save(model.state_dict(), "model.pt")

Epoch 1/10: Loss=0.0002
Epoch 2/10: Loss=0.0012
Epoch 3/10: Loss=0.0004
Epoch 4/10: Loss=0.0019
Epoch 5/10: Loss=0.0101
Epoch 6/10: Loss=0.0065
Epoch 7/10: Loss=0.0062
Epoch 8/10: Loss=0.0111
Epoch 9/10: Loss=0.0053
Epoch 10/10: Loss=0.0063


### Evaluation: Once the model is trained, you can use it to predict the sentiment of the captions in the testing dataset. To evaluate the performance of the model, you will need to compare its predictions to the ground-truth sentiment labels using metrics such as accuracy, precision, recall, and F1-score.

In [23]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

model = MultimodalModel(image_dim=512, text_dim=300, hidden_dim=128)
model.load_state_dict(torch.load("model.pt"))

dataset = TensorDataset(test_caption_image, test_caption_word, torch.tensor(test_caption["sentiment"].values))
batch_size = 100
val_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# evaluate on validation set
with torch.no_grad():
        val_loss = 0
        num_correct = 0
        total_samples = 0
        all_predictions = []
        all_labels = []
        for image, text, label in val_data_loader:
            output = model(image, text)
            val_loss += criterion(output, label.float()).item()
            predictions = (output > 0.5).long()
            num_correct += (predictions == label).sum().item()
            total_samples += label.size(0)
            all_predictions += predictions.tolist()
            all_labels += label.tolist()
        accuracy = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions)
        recall = recall_score(all_labels, all_predictions)

print("accuracy: ",accuracy)
print("precision: ",precision)
print("recall: ",recall)
print("Predictions:", all_predictions)
print("Labels:", all_labels)

accuracy:  0.5727752639517345
precision:  1.0
recall:  0.22050223598211213
Predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Model Improvement: If the performance of the model is not satisfactory, you can iterate and improve it by tweaking the model architecture or adjusting the model's parameters.

In [27]:
# define model
model = MultimodalModel(image_dim=512, text_dim=300, hidden_dim=256)

# define loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

dataset = TensorDataset(train_caption_image,train_caption_word,train_caption_label)
batch_size = 500
dataloader = DataLoader(dataset, batch_size=batch_size)
num_epochs=15

# train loop
for epoch in range(num_epochs):
    for image, text, label in dataloader:
        optimizer.zero_grad()

        # forward pass
        output = model(image, text)

        # calculate loss
        loss = criterion(output, label.float())

        # backward pass and optimization
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}: Loss={loss:.4f}")
torch.save(model.state_dict(), "model.pt")

Epoch 1/15: Loss=0.0559
Epoch 2/15: Loss=0.0703
Epoch 3/15: Loss=0.6159
Epoch 4/15: Loss=0.1675
Epoch 5/15: Loss=0.3781
Epoch 6/15: Loss=0.2604
Epoch 7/15: Loss=0.2688
Epoch 8/15: Loss=0.2161
Epoch 9/15: Loss=0.1808
Epoch 10/15: Loss=0.1638
Epoch 11/15: Loss=0.1470
Epoch 12/15: Loss=0.1330
Epoch 13/15: Loss=0.1077
Epoch 14/15: Loss=0.1816
Epoch 15/15: Loss=0.1862


In [28]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

model = MultimodalModel(image_dim=512, text_dim=300, hidden_dim=256)
model.load_state_dict(torch.load("model.pt"))

dataset = TensorDataset(test_caption_image, test_caption_word, torch.tensor(test_caption["sentiment"].values))
batch_size = 100
val_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# evaluate on validation set
with torch.no_grad():
        val_loss = 0
        num_correct = 0
        total_samples = 0
        all_predictions = []
        all_labels = []
        for image, text, label in val_data_loader:
            output = model(image, text)
            val_loss += criterion(output, label.float()).item()
            predictions = (output > 0.5).long()
            num_correct += (predictions == label).sum().item()
            total_samples += label.size(0)
            all_predictions += predictions.tolist()
            all_labels += label.tolist()
        accuracy = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions)
        recall = recall_score(all_labels, all_predictions)

print("accuracy: ",accuracy)
print("precision: ",precision)
print("recall: ",recall)
print("F1 score: ",f1)
print("Predictions:", all_predictions)
print("Labels:", all_labels)

accuracy:  0.9396681749622926
precision:  0.9434350359958862
recall:  0.9466804265565876
F1 score:  0.9450549450549449
Predictions: [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 